<a href="https://colab.research.google.com/github/ldejuan/covid/blob/master/descriptive_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import zipfile

In [0]:
#read zip train and test data set from zip Archive
PATH_DATA= "./drive/My Drive/covid/"
fileNameZip = 'covid19-global-forecasting-week-1.zip'
archiveName = "%s%s"%(PATH_DATA,fileNameZip)
trainName= 'train.csv'
testName = 'test.csv'
submissionNane = 'submission.csv'
if zipfile.is_zipfile(archiveName):
  with zipfile.ZipFile(archiveName) as archive:
    dfTrain = pd.read_csv(archive.open(trainName,'r')).\
      rename(columns={"Country/Region":"Country_Region",
                      "Province/State":"Province_State"})
      
    dfTest  = pd.read_csv(archive.open(testName,'r')).\
      rename(columns={"Country/Region":"Country_Region",
                      "Province/State":"Province_State"})
    dfSoubmission  = pd.read_csv(archive.open(submissionNane,'r'))
  

In [0]:
dfTrain.head()

In [0]:
dfTest.head()

In [0]:
dfSoubmission.head()

In [0]:
# Recover the number of dates per Province_State
dfTrain[['Country_Region','Province_State','Date','Fatalities']]\
  .groupby(['Country_Region','Province_State']).agg(['min','max','count'])

In [0]:
#Create a new columns with the sum of Fatalities and ConfirmedCases by Country_Region

dfCountryAgg = dfTrain[["Country_Region","Date","Fatalities","ConfirmedCases"]]\
  .groupby(["Country_Region","Date"]).agg('sum')\
  .reset_index()
dfCountryAgg

In [0]:
# Some Graph per Country/Region
selectedCountry = 'Belgium'
dfSelectedCountry = dfCountryAgg.query("Country_Region == @selectedCountry")
dfSelectedCountry[['Date','ConfirmedCases',"Fatalities"]].plot()